# Prereading

Familiarize yourself with the following.


## Antithetic Sampling

Suppose estimator $\hat \theta_1$ and $\hat \theta_2$ have $E[\hat \theta_1] = E[\hat \theta_2] = \theta$ and correlation $\rho = \frac{Cov(\hat \theta_1, \hat \theta_2)}{\sigma_{\hat \theta_1}\sigma_{\hat \theta_2}}$. 

If (for simplicity of demonstration) $\sigma_{\hat \theta_1}^2=\sigma_{\hat \theta_2}^2=\frac{\sigma^2}{n}$ then 

$$
\begin{align*}
\text{Var}\left(\frac{\hat \theta_1+\hat \theta_2}{2}\right) = {} & \frac{\text{Var}(\hat \theta_1)+\text{Var}(\hat \theta_2)}{4} + \frac{2\text{Cov}(\hat \theta_1, \hat \theta_2)}{4} \\
= {} & \frac{\sigma^2}{2n} + \frac{\frac{\sigma}{\sqrt{n}}\frac{\sigma}{\sqrt{n}}\rho}{2} = \frac{(1+\rho)\sigma^2}{2n}
\end{align*}$$

If $\rho<0$ then 
$\text{Var}\left(\frac{\hat \theta_1+\hat \theta_2}{2}\right) = \frac{(1+\rho)\sigma^2}{2n}$
is less than $\text{Var}(\hat \theta_1)$ and $\text{Var}(\hat \theta_2)$.



Achieving $\rho<0$ (and potentially a substantially lower estimation variance) can in fact be easy... 

For example, if $\hat \theta_k = g(\mathbf{x})$ is a function of $x_i$ such that $\rho_{\hat \theta_k,x_i} > 0$

- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = -x_{i} \text{ for } x_{i}\sim N(\mu, \sigma^2_x)$
- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = -x_{i} \text{ for } x_{i}\sim U(0,1)$
- $\rho_{\hat \theta_1,\hat \theta_2} < 0 \text{ if } x_{i}' = F_X^{-1}(1-F_X(x_{i})) \text{ for } x_{i}\sim f(X)$


In [ ]:
# code to be added later today Jan 19 -- last minute finalizations ongoing

# Lecture

First hour of class

## Bagging 

**Antithetic sampling** is somewhat similar to **Bagging** in ***Random Forests*** where for $k=1,\cdots, K$ tree-based precictions $\hat t_k$ have in expectation the same common variance $\sigma^2_{t}$ and shared correlation $\rho = \frac{Cov(t_k,t_{k'})}{\sigma^2_{t}} > 0$. Then a generalization of
$\text{Var}\left(\frac{\hat t_1+\hat t_2}{2}\right) = \frac{\text{Var}(\hat t_1)+\text{Var}(\hat t_2)}{4} + \frac{2\text{Cov}(\hat t_1+\hat t_2)}{4}$ to $K$ rather than $2$

$$
Var(\bar t) = \underbrace{\frac{1}{K^2} \sum_{k=1}^K \sigma^2_t}_{\textrm{sum of variances}} + \underbrace{\frac{2}{K^2} \sum_{k,k'} \sigma_t^2 \rho}_{\textrm{sum of covariances}} = \frac{\sigma^2}{K} + \frac{\frac{2}{2}(K^2-K)\sigma_t^2\rho}{K^2}
 = \rho \sigma^2 + \frac{(1-\rho)\sigma^2}{K}
$$

which means there are benefits to had from averaging even positively correlated estimators.

## Control Variates

$
\large
\begin{align*}
[\hat \theta + \beta (\hat \lambda - \lambda)]\\
Var[\hat \theta + \beta (\hat \lambda - \lambda)] &={}
Var[\hat \theta] + \beta^2Var[\hat \lambda] + 2\beta Cov(\hat \theta, \hat \lambda)\\
& \frac{d}{d\beta}{}
Var[\hat \theta] + \beta^2 Var[\hat \lambda] + 2 \beta Cov(\hat \theta, \hat \lambda) \;\; = \;\; 0 \\
&={} 2\beta Var[\hat \lambda] + 2 Cov(\hat \theta, \hat \lambda) \quad \Longrightarrow \quad \beta \quad = \quad - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]} \\
\hat \theta - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]} (\hat \lambda - \lambda)\\
Var[\hat \theta + \beta (\hat \lambda - \lambda)] &={}
Var[\hat \theta] - \frac{Cov(\hat \theta, \hat \lambda)}{Var[\hat \lambda]}^2 < Var[\hat \theta] \\\\\textbf{Importance Sampling}\\
Cov(\hat \theta, \hat \lambda) &={} Cov\left(\frac{1}{n}\sum_i x_iw^*_i, \frac{1}{n}\sum_i w^*_i\right) \\
&={} \sum_i Cov\left( \frac{1}{n} x_iw^*_i, \frac{1}{n} w^*_i\right) = \frac{1}{n} Cov\left(x_iw^*_i, w^*_i\right) \\
\end{align*}
$


# Class Coding Challenge

This is
- in person in approximately the final two hours of class
- open notes and open internet (including AI chat bot support tools) **but you may not collaborate with classmates**

This will 

- utilize the coding skills practiced in the prelecture homework
- extend the considerations practiced in the prelecture homework
- address topics from the prelecture reading and lectures itself

and is cumulative in nature relative to all content of the course.